# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mne.io.snirf import read_raw_snirf
from get_events_from_snirf import get_events_from_snirf
import itertools

In [2]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

In [45]:
from give_channels import give_channels

# Load Data

In [3]:
path_to_snirf = "../data_hojjat/dec_nef/2/test_001_S001_7d1123e_5.snirf"
snirf = read_raw_snirf(path_to_snirf)

Loading /home/kernel/Desktop/py_test/project/data_hojjat/dec_nef/2/test_001_S001_7d1123e_5.snirf


/home/kernel/Desktop/py_test/project/venv/lib/python3.8/site-packages/mne/externals/pymatreader/pymatreader.py:34: DeprecationWarning: Please use `get_matfile_version` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.miobase` namespace is deprecated.
  from scipy.io.matlab.miobase import get_matfile_version


In [4]:
snirf.get_data().shape

(4168, 4518)

In [5]:
snirf

<RawSNIRF | test_001_S001_7d1123e_5.snirf, 4168 x 4518 (632.8 s), ~5.6 MB, data not loaded>

In [6]:
data = snirf.get_data()

In [7]:
events_df = get_events_from_snirf(path_to_snirf);
events_df.head(15)

,Timestamp,Event,Duration,Value,Experiment,Stim,Label,File
0,44.671425,StartExperiment,571.234335,1.0,1.0,NaN,na,nan
1,44.721467,Stim,0.000000,1.0,1.0,Horse-15-3-1.Png,Horse,15_3_1.Png
2,45.672347,Stim,0.000000,1.0,1.0,Horse-15-3-2.Png,Horse,15_3_2.Png
3,46.623058,Stim,0.000000,1.0,1.0,Horse-15-3-3.Png,Horse,15_3_3.Png
4,47.573675,Stim,0.000000,1.0,1.0,Bicycle-33-3-1.Png,Bicycle,33_3_1.Png
5,48.507729,Stim,0.000000,1.0,1.0,Bicycle-33-3-2.Png,Bicycle,33_3_2.Png
6,49.458677,Stim,0.000000,1.0,1.0,Spider-5-3-1.Png,Spider,5_3_1.Png
7,50.426131,Stim,0.000000,1.0,1.0,Spider-5-3-2.Png,Spider,5_3_2.Png
8,51.376710,Stim,0.000000,1.0,1.0,Spider-5-3-3.Png,Spider,5_3_3.Png
9,52.327817,Stim,0.000000,1.0,1.0,Shark-16-3-1.Png,Shark,16_3_1.Png


# Some Functions

In [8]:
def get_times(df, key):
    times = df[df.Label == key]['Timestamp'].values;
    return times

In [9]:
for a, b in enumerate(['a', 'b']):
    print(a)

0
1


In [10]:
def get_data(data, channels, times, index, window_size, sample_rate):
    out = np.zeros((len(times), len(channels), window_size * sample_rate))
    for time_index, time in enumerate(times):
        for channel_index, channel in enumerate(channels):
            out[time_index, channel_index, :] = data[channel, int((time+index)*sample_rate): int((time+index+window_size)*sample_rate)]
    return out
    # lst = [data[channel, int((time+index)*sample_rate): int((time+index+window_size)*sample_rate)] for time, channel in itertools.product(times.astype(int), channels)]
    # print(np.shape(lst))
    # arr = np.array(lst)

In [11]:
def data_to_feature(data):
    [times, channels, points] = np.shape(data)
    # features = np.ones((times, channels * 3))
    _mean = np.mean(data, axis = 2)
    _min = np.min(data, axis = 2)
    _max = np.max(data, axis = 2)
    features = np.hstack((_mean, _min, _max))
    # features[:, 3:] = data[:]
    return features

In [12]:
def make_x_y(features):
    shape = np.shape(features)
    X = np.vstack(features)
    y = np.hstack([(i+1) * np.ones(shape[1]) for i in range(len(features))])
    return X, y

In [13]:
def classify(X, y):
    clf = SVC(kernel='linear', C=1, random_state=42)
    # clf = MLPClassifier(random_state=1, max_iter=1000)
    scores = cross_val_score(clf, X, y, cv=5)
    return scores

In [14]:
def get_all_classes(events_df):
    classes = events_df.Label.unique()[1:]
    return classes

In [15]:
def get_all_times(all_classes, events_df):
    all_times_lst = [get_times(events_df, clas) for clas in all_classes]
    all_times_array = np.array(all_times_lst, dtype=object)
    return all_times_array

In [16]:
def get_all_data(data, channels, all_times, index, window_size, sample_rate):
    all_data_lst = [get_data(data, channels, times, index, window_size, sample_rate) for times in all_times]
    all_data_arr = np.array(all_data_lst)
    return all_data_arr

In [17]:
def get_all_feature(data):
    all_feature_lst = [data_to_feature(data) for data in all_data]
    all_feature_arr = np.array(all_feature_lst)
    return all_feature_arr

# Run

In [36]:
all_classes = get_all_classes(events_df)[:2]
all_times = get_all_times(all_classes, events_df)

In [37]:
snirf.get_data().shape

(4168, 4518)

In [38]:
all_classes

array(['Horse', 'Bicycle'], dtype=object)

In [39]:
channels = [0], #, 3, 4, 6, 8, 16, 17, 18, 19, 20, 21, 22, 23]#, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
index = 0
window_size = 1
sample_rate=7
all_data = get_all_data(data, channels, all_times, index, window_size, sample_rate)

In [40]:
all_feature = get_all_feature(all_data)

In [41]:
all_data.shape

(2, 15, 1, 7)

In [42]:
all_feature.shape

(2, 15, 3)

In [44]:
X.shape

(30, 3)

In [43]:
print('i', 'w', 's')
import warnings

warnings.simplefilter('ignore')
for index in range(10):
    all_data = get_all_data(data, channels, all_times, index, window_size, sample_rate)
    all_feature = get_all_feature(all_data)
    X, y = make_x_y(all_feature)
    scores = classify(X, y)
    print(index, window_size, 100 * (scores.mean()))

i w s
0 1 66.66666666666667
1 1 63.33333333333333
2 1 56.666666666666664
3 1 53.333333333333336
4 1 53.333333333333336
5 1 43.33333333333333
6 1 46.666666666666664
7 1 49.999999999999986
8 1 53.333333333333336
9 1 50.0


In [ ]:
X, y = make_x_y(all_feature)

In [ ]:
classify(X, y)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
bee_times = get_times(events_df, 'Bee')
key_times = get_times(events_df, 'Key')

In [ ]:
channel_names = [x['ch_name'] for x in snirf.info['chs']]
is_hbo = np.array([1 if channel_name.endswith(
    'HbO') else 0 for channel_name in channel_names], dtype=bool)
channel_locations_3d = [x['loc'][:3] for x in snirf.info['chs']]

In [ ]:
data = snirf.get_data()

In [ ]:
data.shape

In [ ]:
ax, fig = plt.subplots(figsize=(15, 10))
for bee in bee_times.astype(np.int):
    fig.plot(data[0, bee*7:(bee+1)*7])


In [ ]:
window_size = 1
sampling_rate = 7
channels = [0, 1]
index= 6
bee_data = get_data(data, channels, bee_times, index, window_size, sampling_rate)
key_data = get_data(data, channels, key_times, index, window_size, sampling_rate)

In [ ]:
bee_data.shape

In [ ]:
plt.plot(np.mean(bee_data, axis=0))
plt.plot(np.mean(key_data, axis=0))

In [ ]:
bee_features = data_to_feature(bee_data)
key_features = data_to_feature(key_data)

In [ ]:
bee_features.shape

In [ ]:
key_features.shape

In [ ]:
np.hstack([i * np.ones(3) for i in range(3)])

In [ ]:
clf = SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
scores.mean()

In [ ]:
clf = make_pipeline(StandardScaler(),
                    SVC(gamma='auto'))
                    # SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X, y)

In [ ]:
clf.score(X, y)

In [ ]:
len(bee_features)